# XGboost分享

据kaggle在2015年的统计，在29只冠军队中，有17只用的是XGBoost，其中有8只只用了XGBoost。

### 目录
1. 决策树
2. 提升方法(AdaBoost)
3. 什么是XGboost

<center><h1>决策树</h1></center><br>
    决策树是一种基本的分类和回归方法，<br> 决策树模型呈树形结构，在分类问题中, 表示基于特征对实例进行分类的过程，<br>他可以认为是 $if-then$规则的集合。

### 目录
1. 算法步骤
2. 学习过程
3. CART回归树

### 算法步骤
1. 选择一个最优特征（使得各个子集有一个在当前条件下最好的分类），然后按照该特征划分数据集。
2. 如果这些子集能够基本正确分类，则构建叶节点
3. 否则，对这些子集继续选择新的最优特征，然后继续分割
4. 循环往复，直到所有训练数据子集已经被基本正确分类，或没有合适的特征为止

### 看图理解

<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fwyo4ls7euj31jo0laane.jpg"/>

### 决策树学习过程
1. 特征选择
2. 决策树的生成（考虑局部最优）
3. 决策树的剪枝（考虑全局最优,避免过拟合）<br> 损失函数$C_\alpha=C(T)+\alpha|T|$ <br>
$C(T)$是模型对训练数据的误差 $|T|$是模型的复杂度

### 决策树生成算法
1. ID3 (信息增益)
2. C4.5 (信息增益比) <br>
信息增益(熵变)越大 分类能力越强
3. **CART** (分类和回归树)
    1. **回归树**：平方误差最小化准则 $min_{j,s}[min_{c1}\sum_{x_i\in R_1(j,s)}(y_i-c_1)^2+min_{c2}\sum_{x_i\in R_2(j,s)}(y_i-c_2)^2]$
    2. 分类树：基尼指数 $Gini(p) = \sum_{k=1}^{K}p_{k}(1-p_{k}) = 1 - \sum_{k=1}^K p_k^2$

### 数据读取

In [9]:
import pandas as pd
import numpy as np
data = pd.read_csv("./data/1.csv")

In [17]:
data.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
age,youth,youth,youth,youth,youth,middle,middle,middle,middle,middle,old,old,old,old,old
work,1,1,2,2,1,1,1,2,1,1,1,1,2,2,1
house,1,1,1,2,1,1,1,2,2,2,2,2,1,1,1
credit,general,good,good,general,general,general,good,good,super,general,super,good,good,super,general
label,0,0,1,1,0,0,0,1,1,0,1,1,1,1,0


### 数据清理

In [11]:
def init_data(data):
    key_map={
        "youth":1, "middle": 2, "old": 3,
        "general":1, "good":2, "super":3,
    }
    input_data = []
    for i in range(15):
        temp = np.reshape(np.array(data[i:i+1]), 5)[:-1]
        temp[0], temp[3] = key_map[temp[0]], key_map[temp[3]]
        input_data.append(temp)
    labels = data['label']
    return input_data, labels

### 使用sklearn 

In [12]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='gini')
X,Y = init_data(data)
clf.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
clf.predict(X)

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])

In [14]:
np.array(list(Y))

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])

<center>生成决策树</center>
<hr>
<div style="text-align:center;"><img src="http://ww1.sinaimg.cn/large/006r0i4lly1fwzdrbjn0hj311m0ikgnd.jpg"/></div>

### CART 回归树生成
1. 设数据集为$D=\{(x_1,y_1),(x_2,y_2)\dots(x_n,y_n)\}$,设树已有$M$个节点，<br>
$R_1,R_2,\dots,R_M$,每个节点对应一个输出值$c_m$,那么回归树的模型可以表示为<br>
$f(x)=\sum^M_{m=1}c_mI(x\in R_m)$
2. $c_m$取值为$\hat{c}_m=ave(y_i|x_i\in R_m)$ (平方误差准则$\sum_{x_i\in R_m}(y_i-f(x_i))^2$)
3. 对输入空间进行划分 选择切分变量$x^{j}$和他的取值切分点$s$ 并定义区域 $R_{1}(j,s)=\{x|x^{j} \leq s\}$ 和 区域 $R_{2}(j,s)=\{x|x^{j} > s\}$ <br>
求解$min_{j,s}[min_{c1}\sum_{x_i\in R_1(j,s)}(y_i-c_1)^2+min_{c2}\sum_{x_i\in R_2(j,s)}(y_i-c_2)^2]$ <br>
遍历变量$j$,使得上式最小，然后对划分后的区域继续遍历
4. 对划分后的区域再次执行(3) 直到结束

<center><h2>提升方法(AdaBoost)</h2></center> <br>
就是从弱学习算法出发反复学习，得到一系列<strong>弱分类器</strong> ,又称为<strong>基本分类器</strong> ，<br>
然后组合这些弱分类器，构成一个强分类器。<br>
最具代表性的算法<strong>AdaBoost算法</strong>

### 目录
1. AdBoost算法
2. 提升树
3. GBDT

### AdBoost算法

**方法=模型+策略(损失函数)+算法** <br>
二类分类算法(AdBoost) = 加法模型 + 指数函数 + 前向分步算法

### 加法模型
最终的结果是如下的加法模型 <br>
$\begin{equation}
f(x)=\sum^M_{m=1}\beta_mb(x;\gamma_m)
\label{eq:plus_model}
\\ b(x;\gamma_m)是基函数，\gamma_m基函数的参数，\beta_m是基函数的系数。
\end{equation}$ <br>

### 损失函数
当给定损失函数后，问题就转换为使损失函数最小<br>
$\begin{equation}
min_{\beta_m, \gamma_m}\sum^N_{i=1}L(y_i, \sum^M_{m=1}\beta_m b(x_i;\gamma_m))
\label{eq:loss_func}
\end{equation}$ <br>
直接求解比较复杂，使用前向分步算法来解决

### 前向分步算法

前向分步算法的思路是：因为学习的是加法模型，若能从前到后，每一步只学习一个基函数及其系数，逐步逼近优化目标函数，那么就可以简化复杂度。<br>
每步只需要优化如下损失函数 <br>
$min_{\beta, \gamma}\sum^N_{i=1}L(y_i, \beta b(x_i;\gamma))$

<center><img src="http://ww1.sinaimg.cn/large/006r0i4lly1fwzuijf014j30gl0aa0ta.jpg"/></center>

针对于Boosting方法，就有两个地方需要注意：

1. 每一轮如何改变训练数据的权值和概率分布
2. 如何将弱分类器组合成一个强分类器

以AdaBoost算法为例，<br>
对于第1个问题，AdaBoost的方法是提升被前一轮弱分类器错误分类的样本的权值，降低被正确分类样本的权值；<br>
对于第2个问题，AdaBoost采取加权多数表决的方法，加大分类误差率小的弱分类器的权值，减少分类误差率大的弱分类器的权值。

### 提升树
提升树是以分类树或回归树为基础分类器的提升方法，提升树模型可以表示为决策树的加法模型,下面以损失为平方误差为例:<br>
$L(y,F(x))=\frac{1}{2}(y-F(x))^2$ <br>
损失函数的一阶导数<br>
$\frac{\partial L(y,F(x_i))}{\partial F(x_i)} = F(x_i) - y$

### 回归问题的提升树算法
输入: 数据集为$D=\{(x_1,y_1),(x_2,y_2)\dots(x_n,y_n)\}$ <br>
输出: 提升树 $f_M(x)$ <br>
(1) 初始化$f_0(x)=0$ <br>
(2) 对$m=1,2,...,M$ <br> 
    (a) 计算**残差** <br>$r_{mi}=y_i-f_{m-1}(x_i)$ <br>
    (b) 拟合残差 $r_{mi}$学习一个回归树，得到$T(x; \Theta_m)$ <br>
    (c) 更新$f_m(x_i)=f_{m-1}(x)+T(x)$ <br>
(3) 得到回归问题的提升树$f_M(x) = \sum_{m=1}^{M}T(x;\Theta_m)$

<center><strong>简单点!</strong></center>
<hr>
<br>
<center><img src="http://ww1.sinaimg.cn/large/006r0i4lly1fwzvlag53lj30g40auglv.jpg"/></center>


<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fx0lv3vgs6j31ks0sm7hk.jpg"/>


### GBDT (梯度提升决策树)

GBDT所采用的也是加法模型和前向分步算法，树的类型则是CART树,loss函数不定,<br>
当损失函数的是平方损失和指数损失函数时，每一步的优化会比较简单，但对一般损失函数来说，往往每一步优化没有那么简单，<br>
所以 **Freidman**提出梯度提升算法，以负梯度代替残差来求解基函数。<br> <br>
$r_{mi} = -[\frac{\partial L(y,f(x_i))}{\partial f(x_i)}]_{f(x)=f(x)_{m-1}}$

<center><h2>XGBoost</h2></center>

### 前情回顾

1. 决策树生成算法CART
2. 回归树和分类树
3. 方法=模型+策略(损失函数)+算法
4. GBDT

### XGBoost和GBDT有什么关联

1. xgboost和GBDT的学习过程都是一样的，都是基于Boosting的思想，先学习前n-1个学习器，然后基于前n-1个学习器学习第n个学习器。(加法模型，前向分步算法)
2. 建树过程都利用了损失函数的导数信息(Gradient),只是大家利用的方式不一样而已。

### XGBoost和GBDT的区别

1. xgboost和GBDT的一个区别在于目标函数上。 在xgboost中，损失函数+正则项。 GBDT中，只有损失函数。 
2. xgboost中利用二阶导数的信息，而GBDT只利用了一阶导数。 
3. xgboost在建树的时候利用的准则来源于目标函数推导，而GBDT建树利用的是启发式准则。(牛逼之处)

###  目标函数

$Obj(\Theta)=L(\Theta)+\Omega(\Theta)$ <br>
损失函数用于描述模型拟合数据的程度。 <br>
正则项用于控制模型的复杂度。<br>
我们可以利用Boosting的思想来解决这个问题，我们把学习的过程分解成先学第一颗树，然后基于第一棵树学习第二颗树 <br>
$\hat{y}_i^K=\hat{y}_i^{K-1}+f_K(x_i)$<br>
所以对第K个目标函数可得<br>
$Obj^K=\sum_iL(y_i,\hat{y}_i^K)+\Omega(f_K)+constant$ <br>
$Obj^K=\sum_iL\left(y_i,\hat{y}_i^{K-1}+f_K(x_i)\right)+\Omega(f_K)+constant$ <br>

### 二阶泰勒展开

$f(x+\Delta x)=f(x)+f'(x)\Delta x+\frac{1}{2}f''(x){\Delta x}^2$<br>
对损失函数进行二阶泰勒展开 其中我们把$\hat{y_i}^{K-1}$看做$x$，把$f_K(x_i)$看做$\Delta x$, 就能转换成下面的式子
$\sum_iL\left(y_i,\hat{y_i}^{K-1}+f_K(x_i)\right)=\sum_i\left[L(y_i,\hat{y}_i^{K-1})+L'(y_i,\hat{y}_i^{K-1})f_K(x_i)+\frac{1}{2}L''(y_i,\hat{y}_i^{K-1})f_K^2(x_i)\right]$ <br>
然后下面再加上正则项$\Omega(f_t) = \gamma T + \frac 1 2 \lambda \sum_{j = 1}^T w_j^2$进行化简整理

<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fx0oxcxe9qj30zk0k00w2.jpg"/>

### 几个简单的替代
针对式子 $\sum_iL\left(y_i,\hat{y_i}^{K-1}+f_K(x_i)\right)=\sum_i\left[L(y_i,\hat{y}_i^{K-1})+L'(y_i,\hat{y}_i^{K-1})f_K(x_i)+\frac{1}{2}L''(y_i,\hat{y}_i^{K-1})f_K^2(x_i)\right]$ <br>
我们用$g_i$代替$f^{\prime}(x)$, 用$h_i$代替$f^{\prime \prime}(x)$, 因为在第k步时，$\hat{y}^{k-1}_i$是已知值，所以$l(y_i,\hat{y}^{k-1}_i)$是常数，不影响函数优化，可以省去.<br>
所以我们的目标函数可以表示为<br>
$Obj^{(k)} \approx \sum_{i = 1}^n [g_i f_k(x_i) + \frac 1 2 h_i f_k^2 (x_i)] + \Omega (f_k)$ <br> 在这里变量是$f_k(x)$就是我们要学习的决策树

### $f_t(x)$
对$f_t(x)$进行转换一下 <br>
$f_t(x)=w_{q(x)} \\
q(x)代表样本x位于哪个叶子结点\\
w_q代表该叶子结点的取值$

<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fx0usaizfhj315i0l2n52.jpg"/>

### 正则项
用叶子的个数和叶子权重的平滑程度来描述模型的复杂度 <br>
$\Omega(f_t) = \gamma T + \frac 1 2 \lambda \sum_{j = 1}^T w_j^2$<br>
用$I_j$来表示第$j$个叶子节点的样本集合，就是下图中的每个圆圈中的人。

<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fx0v2c6um9j31d80mmdmn.jpg"/>

## 目标函数化简
\begin{align}
Obj^{(t)} &\approx \sum_{i = 1}^n [g_i f_t(x_i) + \frac 1 2 h_i f_t^2 (x_i)] + \Omega (f_t) \\
          &= \sum_{i = 1}^n [ g_i w_{q(x_i)} + \frac 1 2 h_i w_{q(x)}^2] + \gamma T + \frac 1 2 \lambda \sum_{j = 1}^T w_j^2 \\
          &= \sum_{j = 1}^T [(\sum_{i \in I_j } g_i)w_j + \frac 1 2 (\sum_{i \in I_j}h_i + \lambda)w_j^2] + \gamma T
\end{align}

我们再进行替换 $G_j=\sum_{(i \in I_j)}g_i$ $H_j=\sum_{(i \in I_j)}h_i$<br>
然后最终化简结果为:<br>
$\sum_{j=1}^{T}\left[G_jw_j+\frac{1}{2}(H_j+\lambda )w_{j}^2\right]+\gamma T $<br>
假设树的结构已知，即$q(x)$已知 ，那么对目标函数求一阶导数，然后代入极值点 <br>
$w^*=-\frac{G_j}{H_j+\lambda}$<br>
$Obj=-\frac{1}{2}\sum_{j=1}^T\frac{G_j^2}{H_j+\lambda}+\gamma T$

<img src="http://ww1.sinaimg.cn/large/006r0i4lly1fx0vl0151zj31j20p4jzh.jpg"/>

### 树的分裂
XGboost 分裂准则是直接与损失函数挂钩的准则<br>
$Gain=\frac{1}{2}\left[\frac{G_L^2}{H_L^2+\lambda}+\frac{G_R^2}{H_R^2+\lambda}-\frac{(GL+G_R)^2}{(H_L+H_R)^2+\lambda}\right]-\gamma $ <br>
$\begin{align}
    Obj_{split} &= - \frac 1 2 [\frac {G_L^2}{H_L + \lambda} + \frac {G_R^2}{H_R + \lambda}] + \gamma T_{split} \\
    Obj_{noSplit} &=  - \frac 1 2 \frac {(G_L + G_R)^2}{H_L + H_R + \lambda} + \gamma T_{noSplit} \\
    Gain &= Obj_{noSplit} - Obj_{split} \\
         &= \frac 1 2 [\frac {G_L^2}{H_L + \lambda} + \frac {G_R^2}{H_R + \lambda} - \frac {(G_L + G_R)^2}{H_L + H_R + \lambda}] - \gamma(T_{split} - T_{nosplit})
\end{align}$

其实我们已经把xgboost的基本原理阐述了一遍。我们总结一下，就是做了以下几件事情。 
1. 在损失函数的基础上加入了正则项。 
2. 对目标函数进行二阶泰勒展开。 
3. 利用推导得到的表达式作为分裂准确，来构建每一颗树。

[GBDT原理与Sklearn源码分析-回归篇](https://blog.csdn.net/qq_22238533/article/details/79185969)<br>
[xgboost原理分析以及实践](https://blog.csdn.net/qq_22238533/article/details/79477547)<br>
[机器学习-一文理解GBDT的原理](https://zhuanlan.zhihu.com/p/29765582)<br>
**图片来自陈天齐的论文ppt**